# Data Pre-Processing

## Dealing with [*Missing* Data](https://en.wikipedia.org/wiki/Missing_data)

In most online _machine learning_ classes you are taught that when your data set is **incomplete** you can either:
1. Erase the corresponding rows with missing cells; or
2. _Impute_ (fill) the sample average of each column into those missing cells.

It turns out that the second option would require stronger assumptions than the first. If the observations are **missing completely at random** and your sample is i.i.d., the first option is harmless for large data sets.

## Encoding [Categorical Variables](https://en.wikipedia.org/wiki/Categorical_variable)

💻 Consider the ```hprice3``` data set from the ```wooldridge``` package:

In [ ]:
## installing the 'wooldridge' package if not previously installed
if (!require(wooldridge)) install.packages('wooldridge')

data(hprice3)

## Obs:   321

##  1. year                     1978, 1981
##  2. age                      age of house
##  3. agesq                    age^2
##  4. nbh                      neighborhood, 1 to 6
##  5. cbd                      dist. to central bus. dstrct, feet
##  6. inst                     dist. to interstate, feet
##  7. linst                    log(inst)
##  8. price                    selling price
##  9. rooms                    # rooms in house
## 10. area                     square footage of house
## 11. land                     square footage lot
## 12. baths                    # bathrooms
## 13. dist                     dist. from house to incin., feet
## 14. ldist                    log(dist)
## 15. lprice                   log(price)
## 16. y81                      =1 if year = 1981
## 17. larea                    log(area)
## 18. lland                    log(land)
## 19. linstsq                  linst^2

In [3]:
import wooldridge as woo
hprice3 = woo.dataWoo('hprice3')

💻 Variables ```y81```, ```rooms```, and ```nbh``` are examples of <ins>categorical</ins> variables. In Econometrics, ```y81``` is called a standard dummy variable, ```rooms``` is called an _ordered_ categorical variable, ```nbh``` is called an _unordered_ categorical variable. Both ```rooms``` and ```nbh``` have _multiple_ categories. The fucntion ```as.factor()``` with option ```ordered=TRUE``` and ```ordered=FALSE``` (default) will allow us to handle them accordingly in all analysis.

In [ ]:
## without using the 'as.factor' function
attach(hprice3)
no.factor <- data.frame(y81=y81,rooms=rooms,nbh=nbh)
summary(no.factor)
detach(hprice3)

## using the 'factor' function
attach(hprice3)
yes.factor <- data.frame(y81=factor(y81),
                         rooms=factor(rooms,ordered=TRUE),
                         nbh=factor(nbh,ordered=FALSE)
                         )
summary(yes.factor)
detach(hprice3)

In [25]:
import pandas as pd
no_factor = hprice3[['y81','rooms','nbh']]
print(no_factor.dtypes)
yes_factor = pd.DataFrame({'y81':hprice3['y81'],
                           'rooms':hprice3['rooms'].astype('category'),
                           'nbh':hprice3['nbh'].astype('category')})
yes_factor.rooms.cat.set_categories(
    new_categories = sorted(hprice3['rooms'].unique()), ordered = True, inplace = True)
print(yes_factor.dtypes)

y81      int64
rooms    int64
nbh      int64
dtype: object
y81         int64
rooms    category
nbh      category
dtype: object



**<span style="color:green">Exercise:</span>**: Plot a histogram of the variable ```rooms``` when it is not treated as a factor and when it is treated as an ```ordered=TRUE``` factor


In [ ]:
hist(no.factor$rooms,main="(Non-Categorical) Rooms",xlab="",ylab="")
plot(yes.factor$rooms,main="(Ordered Categorical) Rooms",xlab="",ylab="")

💻 The default behavior in regression is to transformed ordered and unordered categorical variable with multiple categories into a set of $c-1$ dummy variables and include them as regressors, where $c$ represents the number of categories.

In [ ]:
ols <- lm(lprice ~ lland + larea + I(log(cbd)) +
                   as.factor(y81) + as.factor(rooms) + as.factor(nbh) +
                   linst + linstsq + ldist + baths + age + agesq,
          data=hprice3)

## installing the 'lmtest', 'sandwich' packages if not previously installed
if (!require(lmtest)) install.packages('lmtest')
if (!require(sandwich)) install.packages('sandwich')

## turning 'off' scientific notation
options(scipen = 999) 

## calculating standard t-statistics for 'significance'
coeftest(ols, vcov = vcovHC, type = "HC1")

💻 In many machine learning algorithms you are required to provide the design (model) matrix, $\mathbf{X}$ (*without* and intercept), and response vector, $\mathbf{y}$.

**<span style="color:green">Exercise:</span>**: Use the ```model.matrix``` function to extract the design matrix _without_ an intercept from the previously created ```ols``` object and verify it contains 22 columns of regressors/features.

In [ ]:
X <- model.matrix(ols)[,-1]
dim(X)
colnames(X)

📌 It is good practice to define categorical variables _outside_ the model formula/fitting. When doing this, one can easily change the 'base' category using the ```relevel()``` function along with the ```within()``` function.

**<span style="color:green">Exercise:</span>**:
1. Create a copy of the ```hprice3``` data frame, call it ```hprice3.copy``` that contains all the variables used in fitting the model ```ols``` above.

In [ ]:
hprice3.copy <- subset(hprice3,select=c("lprice","lland","larea","cbd",
                                        "y81","rooms","nbh","linst",
                                        "linstsq","ldist","baths","age","agesq"))
names(hprice3.copy)

2. Add the variable ```lcbd``` which equals the natural logarithm of ```cbd``` and then drop ```cbd``` from ```hprice3.copy```.

In [ ]:
hprice3.copy$lcbd <- log(hprice3.copy$cbd)
hprice3.copy <- subset(hprice3.copy,select=-c(cbd))
names(hprice3.copy)

3. Replace ```y81```, ```rooms```, and ```nbh``` by their factor versions in ```hprice3.copy```.

In [ ]:
hprice3.copy$y81 <- factor(hprice3.copy$y81)
hprice3.copy$rooms <- factor(hprice3.copy$rooms,ordered=TRUE)
hprice3.copy$nbh <- factor(hprice3.copy$nbh,ordered=FALSE)
head(hprice3.copy)

4. Make ```y81==1``` and ```nbh==3``` the base categories in ```hprice3.copy```

In [ ]:
hprice3.copy <- within(hprice3.copy, y81 <- relevel(y81,ref=2))
hprice3.copy <- within(hprice3.copy, nbh <- relevel(nbh,ref=4))

5. Run a regression of ```lprice``` on all the other features in the ```hprice3.copy``` and report the outcome.

In [ ]:
ols.sim <- lm(lprice~.,data=hprice3.copy)
coef(ols.sim)

## Including [Interaction Terms](https://en.wikipedia.org/wiki/Interaction_(statistics) )

In the previously fitted model we included ```linstsq``` and ```agesq``` as predictors. These correspond to the squared of the original predictors ```linst``` and ```age```. In economics we include such predictors to account for increasing/decreasing returns to scale in modelling. Since $\texttt{linst}^2=\texttt{linst}\times\texttt{linst}$ and $\texttt{age}^2=\texttt{age}\times\texttt{age}$, one can think of them as a specific type of interaction terms (products with themselves).

<ins>Example</ins>: Consider the following version of the previously estimated model

$$
\begin{aligned}
\texttt{lprice}&=\beta_0+\beta_1\texttt{lland}+\beta_2\texttt{larea}+ \beta_3\texttt{linst}+\beta_4\texttt{age}\\
               & + \beta_5\texttt{linst}\times\texttt{age}+\beta_6\texttt{linst}^2+\beta_7\texttt{age}^2+e
\end{aligned}
$$

In [ ]:
## OLS: estimating this new model
ols.0 <- lm(lprice ~ lland + larea + linst + age
                     + I(linst*age) + linstsq + agesq,
          data=hprice3)
round(coef(ols.0),5)

By defining $\mathbf{x}=[\texttt{lland},\texttt{larea},\texttt{linst},\texttt{age}]^\prime$, and Assumption MLR.4, one has

$$
\frac{\partial E[\texttt{lprice}|\mathbf{x}]}{\partial \texttt{linst}}=\beta_3+\beta_5\texttt{age}+2\beta_6\texttt{linst}.
$$

Then $\beta_3$ represents the average price elasticty with respect to the distance from the highway for a home that is brand new ($\texttt{age}=0$) and that is located 1 feet away from the highway ($\texttt{linst}=\log(\texttt{inst})=0$, i.e., $\texttt{inst}=1$).

💻 Calculating the summary statistics for variables ```age``` and ```inst``` in the data set.

In [ ]:
## printing summary statistics for 'age' and 'inst'
print(summary(subset(hprice3,select=c("age","inst","linst"))))

Now consider the following alternative specification

$$
\begin{aligned}
\texttt{lprice}&=\delta_0+\beta_1\texttt{lland}+\beta_2\texttt{larea}+ \delta_3\texttt{linst}+\delta_4\texttt{age}\\
               & + \beta_5[\texttt{linst}-\mu_{\texttt{linst}}]\times[\texttt{age}-\mu_{\texttt{age}}]+\beta_6[\texttt{linst}-\mu_{\texttt{linst}}]^2+\beta_7[\texttt{age}-\mu_{\texttt{age}}]^2+e.
\end{aligned}
$$

In this case

$$
\frac{\partial E[\texttt{lprice}|\mathbf{x}]}{\partial \texttt{linst}}=\delta_3+\beta_5[\texttt{age}-\mu_{\texttt{age}}]+2\beta_6[\texttt{linst}-\mu_{\texttt{linst}}].
$$

Then $\delta_3$ represents the average price elasticty with respect to the distance from the highway for a home that is 18 years old ($\widehat{\mu}_{\texttt{age}}\approx 18.01$) and that is located 16,442 feet away from the highway ($\widehat{\mu}_{\texttt{inst}}\approx 16,442$).

In [ ]:
## OLS: estimating this new 'version' model
ols.1 <- lm(lprice ~ lland + larea + linst + age
                     + I((linst-log(mean(inst)))*(age-mean(age)))
                     + I((linst-log(mean(inst)))^2) + I((age-mean(age))^2),
          data=hprice3)

## printing ols results up to 5 decimals
round(data.frame(ols.0=coef(ols.0),ols.1=coef(ols.1)),5)

⁉️ Is it a coincidence that some of the estimates are the same in both specifications? provide an algebraic explanation.

## Beta Coefficients

Why is standardization useful? It is easiest to start with the original OLS equation, with the variables in their original forms:

$$
y_{i}=\widehat{\beta}_{0}+\widehat{\beta}_{1} x_{i 1}+\widehat{\beta}_{2} x_{i 2}+\ldots+\widehat{\beta}_{k} x_{i k}+\hat{e}_{i}.
$$

We have included the observation subscript i to emphasize that our standardization is applied to all sample values. If we average the previous equation and use the fact that $\{\widehat{e}_i:i=1,\ldots,n\}$ has sample mean zero one has

$$
y_{i}-\overline{y}=\widehat{\beta}_{1}\left(x_{i 1}-\overline{x}_{1}\right)+\widehat{\beta}_{2}\left(x_{i 2}-\overline{x}_{2}\right)+\ldots+\widehat{\beta}_{k}\left(x_{i k}-\overline{x}_{k}\right)+\widehat{e}_{i}.
$$

Now, let $\widehat{\sigma}_{y}$ be the sample standard deviation for the dependent variable, let $\widehat{\sigma}_{1}$ be the sample sd for $x_{1}$, let $\widehat{\sigma}_{2}$ be the sample standard deviation, ```sd()```, for $x_{2}$, and so on. Then, simple algebra gives the equation

$$
\begin{aligned}
\left(y_{i}-\overline{y}\right) / \widehat{\sigma}_{y} &=\left(\widehat{\sigma}_{1} / \widehat{\sigma}_{y}\right) \widehat{\beta}_{1}\left[\left(x_{i 1}-\overline{x}_{1}\right) / \widehat{\sigma}_{1}\right]+\ldots \\ &+\left(\widehat{\sigma}_{k} / \widehat{\sigma}_{y}\right) \widehat{\beta}_{k}\left[\left(x_{i k}-\overline{x}_{k}\right) / \widehat{\sigma}_{k}\right]+\left(\widehat{e}_{i} / \widehat{\sigma}_{y}\right).
\end{aligned}
$$

Now define $z_{i y}=\left(y_{i}-\overline{y}\right) / \widehat{\sigma}_{y}$, $z_{i l}=\left[\left(x_{i l}-\overline{x}_{l}\right) / \widehat{\sigma}_{l}\right]$, $\widehat{b}_{l}=\left(\widehat{\sigma}_{l} / \widehat{\sigma}_{y}\right) \widehat{\beta}_{l}$ for $l=1,\ldots,k$, and $\widehat{\varepsilon}_i=\left(\widehat{e}_{i} / \widehat{\sigma}_{y}\right)$. Then

$$
z_{y}=\widehat{b}_{1} z_{1}+\widehat{b}_{2} z_{2}+\ldots+\widehat{b}_{k} z_{k}+\widehat{\varepsilon}_i.
$$

The new coefficients are

$$
\widehat{b}_{j}=\left(\widehat{\sigma}_{j} / \widehat{\sigma}_{y}\right) \widehat{\beta}_{j} \text { for } j=1, \ldots, k.
$$

These $\widehat{b}_{j}$ are traditionally called **standardized coefficients** or **beta coefficients**.

✍🏽 If $x_l$ increases by one standard deviation, then $\widehat{y}$ changes by $\widehat{b}_l$ standard deviations. Thus, *we are measuring effects not in terms of the original units of $y$ or the $x_l$, but in standard deviation units*.

In [ ]:
## installing the 'lm.beta' package if not previously installed
if (!require(lm.beta)) install.packages('lm.beta')

lm.beta(ols.1)

💻 Since each $x_l$ have been standardized, comparing the _magnitudes_ of the resulting beta coefficients is now useful. The ```age``` of the house seems to have the largest effect on the price of a home.

<ins>Pre-Processing in Machine Learning</ins>: In machine learning, standardizing (*re-centering* and *scaling*) predictors is commonly done before model fitting. No transformation is usually done to the outcome variable.

In [ ]:
X1 <- model.matrix(ols.1)[,-1]
datos <- cbind(data.frame(lprice=hprice3$lprice),as.data.frame(X1))

## installing the 'caret' package if not previously installed
if (!require(caret)) install.packages('caret')

modelo <- train(lprice ~ .,
               data = datos,
               method = "lm",
               preProcess = c('scale', 'center')
               )

coef(modelo$finalModel)